## A minimalist example for recovering sparse graphs using `uGLAD`

Fitting uGLAD on a erdos-renyi random sparse graph with samples obtained from a corresponding multivariate Gaussian distribution.    

### About `uGLAD` 
Sparse graph recovery by optimizing deep unrolled networks. This work proposes `uGLAD` which is a unsupervised version of a previous `GLAD` model (GLAD: Learning Sparse Graph Recovery (ICLR 2020 - [link](<https://openreview.net/forum?id=BkxpMTEtPB>)).  

Key benefits & features:  
- Solution to Graphical Lasso: A better alternative to solve the Graphical Lasso problem as
    - The neural networks of the uGLAD enable adaptive choices of the hyperparameters which leads to better performance than the existing algorithms  
     - No need to pre-specify the sparsity related regularization hyperparameters    
    - Requires less number of iterations to converge due to neural network based acceleration of the unrolled optimization algorithm (Alternating Minimization)    
    - GPU based acceleration can be leveraged  
    - Novel `consensus` strategy which robustly handles missing values by leveraging the multi-task learning ability of the model   
- Glasso loss function: The loss is the logdet objective of the graphical lasso `1/M(-1*log|theta|+ <S, theta>)`, where `M=num_samples, S=input covariance matrix, theta=predicted precision matrix`.  
- Ease of usability: Matches the I/O signature of `sklearn GraphicalLassoCV`, so easy to plug-in to the existing code.  

In [1]:
import os, sys
# reloads modules automatically before entering the 
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# install jupyter-notebook in the env if the prefix does not 
# show the desired virtual env. 
print(sys.prefix)
import warnings
warnings.filterwarnings('ignore')

/home/harshx/anaconda3/envs/uGLAD


In [2]:
import torch
torch.__version__

'1.10.1'

### Create sample data

In [3]:
from uGLAD.utils.prepare_data import get_data
from uGLAD.utils.metrics import reportMetrics

# Xb = samples batch, trueTheta = corresponding true precision matrices
Xb, true_theta = get_data(
    num_nodes=10, 
    sparsity=0.2, 
    num_samples=500, 
    batch_size=1,
    eig_offset=1, 
    w_min=0.5,
    w_max=1
)
print(f'true_theta: {true_theta.shape}, Samples {Xb.shape}')

true_theta: (1, 10, 10), Samples (1, 500, 10)


### The uGLAD model

Learning details:  
1. Initialize learnable `GLAD` parameters  
2. Run the GLAD model  
3. Get the glasso-loss  
4. Backprop  

Possible solutions if `uGLAD` does not converge:  
1. Increase number of training EPOCHS
2. Lower the learning rate    
3. Please re-run. This will run the optimization with different initializations  
4. Change the INIT_DIAG=0/1 in the `GLAD` model parameters  
5. Increase `L`, the number of unrolled iterations of `GLAD`

### Running the uGLAD-Direct mode

- Directly optimize the uGLAD model on the complete data X
- Optimizes the model to minimize the glasso-loss on X 

In [4]:
from uGLAD import main as uG

# Initialize the model
model_uGLAD = uG.uGLAD_GL()  

# Fit to the data
model_uGLAD.fit(
    Xb[0],
    centered=False,
    epochs=250,
    lr=0.002,
    INIT_DIAG=0,
    L=15,
    verbose=False, 
    k_fold=0,  # Direct mode
    mode='direct'
)  

# Comparing with true precision matrix
compare_theta_uGLAD = reportMetrics(
        true_theta[0], 
        model_uGLAD.precision_
    )
print(f'uGLAD: {compare_theta_uGLAD}')

Running uGLAD
Direct Mode
Total runtime: 8.951887369155884 secs

uGLAD: {'FDR': 0.2857142857142857, 'TPR': 1.0, 'FPR': 0.11428571428571428, 'SHD': 4, 'nnzTrue': 10, 'nnzPred': 14, 'precision': 0.7142857142857143, 'recall': 1.0, 'Fbeta': 0.8333333333333334, 'aupr': 1.0, 'auc': 1.0}


### Running the uGLAD-CV mode 

- Finds the best model by doing cross-fold validation on the input samples X
- Chooses the model which performs best in terms of glasso-loss on held-out data
- More conservative than the direct mode

In [5]:
from uGLAD import main as uG

# Initialize the model
model_uGLAD = uG.uGLAD_GL()  

# Fit to the data
model_uGLAD.fit(
    Xb[0],
    centered=False,
    epochs=250,
    lr=0.002,
    INIT_DIAG=0,
    L=15,
    verbose=False,
    k_fold=3, 
    mode='cv'
)  

# Comparing with true precision matrix
compare_theta_uGLAD = reportMetrics(
        true_theta[0], 
        model_uGLAD.precision_
    )
print(f'uGLAD: {compare_theta_uGLAD}')

Running uGLAD
CV mode: 3-fold
Total runtime: 36.888649702072144 secs

uGLAD: {'FDR': 0.0, 'TPR': 1.0, 'FPR': 0.0, 'SHD': 0, 'nnzTrue': 10, 'nnzPred': 10, 'precision': 1.0, 'recall': 1.0, 'Fbeta': 1.0, 'aupr': 1.0, 'auc': 1.0}


### Comparison with sklearn's GraphicalLassoCV

In [6]:
from sklearn.covariance import GraphicalLassoCV

model_BCD = GraphicalLassoCV().fit(Xb[0])
# Compare with theta
compare_theta_BCD = reportMetrics(
    true_theta[0], 
    model_BCD.precision_
)
print(f'BCD: {compare_theta_BCD}')

BCD: {'FDR': 0.5833333333333334, 'TPR': 1.0, 'FPR': 0.4, 'SHD': 14, 'nnzTrue': 10, 'nnzPred': 24, 'precision': 0.4166666666666667, 'recall': 1.0, 'Fbeta': 0.5882352941176471, 'aupr': 1.0, 'auc': 1.0}


# Handling missing values
Running `uGLAD` model in mode=`missing`:
- Leverages the multi-task learning feature of the `uGLAD` model
- Uses the novel `consensus` strategy to robustly handle the missing values

In [79]:
# Adding dropout noise to Xb
from uGLAD.utils.prepare_data import add_noise_dropout
from uGLAD.main import mean_imputation
import numpy as np

# Adding np.NaNs to introduce missing values
Xb_miss = add_noise_dropout(Xb, dropout=0.75)
# Doing mean imputation for basic statistical comparsion
B, M, D = Xb_miss.shape
Xb_mean = [] 
for b in range(B):
    X_miss = Xb_miss[b].copy()
    X_miss = X_miss.reshape(1, M, D)
    Xb_mean.append(mean_imputation(X_miss).reshape(M, D))
Xb_mean = np.array(Xb_mean)

In [80]:
np.isnan(np.sum(Xb_miss)), Xb_miss, Xb_mean, Xb

(True,
 array([[[-0.04706259, -0.52160413,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan, -0.0156085 ],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         ...,
         [        nan,         nan,         nan, ...,         nan,
           0.29022626,         nan],
         [        nan,  0.67902615,         nan, ...,         nan,
                  nan,         nan],
         [-0.480543  ,         nan,         nan, ...,         nan,
                  nan,         nan]]]),
 array([[[-0.04706259, -0.52160413,  0.00632773, ...,  0.00339748,
          -0.03011912, -0.01321154],
         [-0.0199985 ,  0.01545049,  0.00632773, ...,  0.00339748,
          -0.03011912, -0.0156085 ],
         [-0.0199985 ,  0.01545049,  0.00632773, ...,  0.00339748,
          -0.03011912, -0.01321154],
         ...,
         [-0.0199985 ,  0.

### Running the `uGLAD-miss` model in missing data mode 

In [ ]:
from uGLAD import main as uG

# Initialize the model
model_uGLAD = uG.uGLAD_GL()  

# Fit to the data
model_uGLAD.fit(
    Xb_miss[0],
    centered=False,
    epochs=500,
    lr=0.003,
    INIT_DIAG=0,
    L=15,
    verbose=False,
    k_fold=5,  # The number of sumsample splits
    mode='missing'
)  

# Comparing with true precision matrix
compare_theta_uGLAD = reportMetrics(
        true_theta[0], 
        model_uGLAD.precision_
    )
print(f'uGLAD: {compare_theta_uGLAD}')

Running uGLAD
Handling missing data
Creating K=5 row-subsampled batches


### Comparison with BCD-mean
Run GrapicalLassoCV with mean imputed Xb_mean

In [81]:
from sklearn.covariance import GraphicalLassoCV

model_BCD = GraphicalLassoCV().fit(Xb_mean[0])
# Compare with theta
compare_theta_BCD = reportMetrics(
    true_theta[0], 
    model_BCD.precision_
)
print(f'BCD: {compare_theta_BCD}')

BCD: {'FDR': 0.36363636363636365, 'TPR': 0.7, 'FPR': 0.11428571428571428, 'SHD': 7, 'nnzTrue': 10, 'nnzPred': 11, 'precision': 0.6363636363636364, 'recall': 0.7, 'Fbeta': 0.6666666666666666, 'aupr': 0.7194444444444443, 'auc': 0.8214285714285714}
